In [17]:
knitr::opts_chunk$set(
  echo = TRUE,
  warning = FALSE,
  message = FALSE,
  fig.width = 10,
  fig.height = 6
)

- ACC (adrenocortical carcinoma)
- UVM (uveal melanoma)
- SKCM (skin cutaneous melanoma)
- LLG (brain lower grade glioma)
- GBM (glioblatoma)


# Setup

In [18]:
# Data manipulation and analysis
library(dplyr)        # Data manipulation and transformation
library(tidyverse)    # Collection of data science packages
library(data.table)   # Fast data manipulation

# Survival analysis
library(survival)     # Core survival analysis functions
library(survminer)    # Survival analysis visualization

# Genomic data handling
library(recount3)     # Access to RNA-seq data
library(biomaRt)      # Access to genomic annotations
library(TCGAbiolinks) # TCGA data access
library(SummarizedExperiment) # Container for genomic data
library(DESeq2)      # RNA-seq analysis
library(GenomicFeatures) # Genomic feature handling
library(rtracklayer)  # Import/export genomic tracks

# Matrix operations
library(matrixStats)  # Matrix calculations
library(sparseMatrixStats) # Sparse matrix operations

# Parallel processing
library(parallel)     # Base R parallel processing
library(BiocParallel) # Bioconductor parallel processing

# Utilities
library(httr)         # HTTP requests
library(retry)        # Retry failed operations
library(futile.logger) # Logging functionality
library(viridis)      # Color palettes

In [19]:
# Set R environment variables for better performance
Sys.setenv(R_MAX_NUM_DLLS = 150)
Sys.setenv(R_GC_MEM_GROW = 3)
Sys.setenv(R_ENABLE_JIT = 3)

# For parallel processing, you can set these based on your available cores
# For example, if you want to use 32 cores like in the shell script:
num_cores <- 32
Sys.setenv(OMP_NUM_THREADS = num_cores)
Sys.setenv(OPENBLAS_NUM_THREADS = num_cores)
Sys.setenv(MKL_NUM_THREADS = num_cores)

# Verify the environment variables were set
Sys.getenv(c("R_MAX_NUM_DLLS", "R_GC_MEM_GROW", "R_ENABLE_JIT", 
             "OMP_NUM_THREADS", "OPENBLAS_NUM_THREADS", "MKL_NUM_THREADS"))

R_MAX_NUM_DLLS        R_GC_MEM_GROW         R_ENABLE_JIT 
               "150"                  "3"                  "3" 
     OMP_NUM_THREADS OPENBLAS_NUM_THREADS      MKL_NUM_THREADS 
                "32"                 "32"                 "32"

In [20]:
#####################################################################
# Set Up Parallel Processing
#####################################################################
# Detect number of cores from SLURM environment or default to 1
num_cores <- as.numeric(Sys.getenv("SLURM_NTASKS", unset = "1"))
if (num_cores > 1) {
  message(sprintf("Setting up parallel processing with %d cores", num_cores))
  BiocParallel::register(MulticoreParam(workers = num_cores))
} else {
  message("Running in single-core mode")
}

Running in single-core mode



In [21]:
#####################################################################
# Create Directory Structure
#####################################################################
# Create directories for storing results, cache, and logs
dir.create("results_notebook", showWarnings = FALSE)  # Store analysis results
dir.create("cache", showWarnings = FALSE)    # Store cached data

# Functions

In [22]:
#####################################################################
# Ensembl Connection Management
#####################################################################
# Cache connection to Ensembl to avoid repeated connections
ensembl <- NULL
get_ensembl_connection <- function() {
  if (is.null(ensembl)) {
    ensembl <<- useMart("ensembl", dataset = "hsapiens_gene_ensembl")
  }
  return(ensembl)
}

In [23]:
#####################################################################
# Clinical Data Retrieval and Processing
#####################################################################
# Function to retrieve and clean clinical data from TCGA
get_clinical_data <- function(cancer_type) {
  message(sprintf("Getting clinical data for %s", cancer_type))
  clinical <- GDCquery_clinic(project = paste0("TCGA-", cancer_type), type = "clinical")
  
  if (nrow(clinical) == 0) {
    stop(sprintf("No clinical data found for %s", cancer_type))
  }
  
  # Print column names for debugging
  message("Available clinical columns:")
  message(paste(colnames(clinical), collapse=", "))
  
  # Clean and standardize clinical data
  clinical_clean <- clinical %>%
    dplyr::transmute(
      case_id = submitter_id,
      vital_status = vital_status,
      # Calculate survival time using either death or last follow-up
      overall_survival = case_when(
        !is.na(days_to_death) ~ as.numeric(days_to_death),
        !is.na(days_to_last_follow_up) ~ as.numeric(days_to_last_follow_up),
        TRUE ~ NA_real_
      ),
      # Standardize vital status to boolean
      deceased = case_when(
        tolower(vital_status) == "dead" ~ TRUE,
        tolower(vital_status) == "alive" ~ FALSE,
        TRUE ~ NA
      )
    ) %>%
    dplyr::filter(!is.na(overall_survival), !is.na(deceased))
  
  # Print summary statistics
  message(sprintf("Processed clinical data summary:"))
  message(sprintf("- Total patients: %d", nrow(clinical_clean)))
  message(sprintf("- Patients with death events: %d", sum(clinical_clean$deceased)))
  message(sprintf("- Median survival time: %.1f days", median(clinical_clean$overall_survival)))
  
  return(clinical_clean)
}

In [24]:
#####################################################################
# Expression Data Retrieval and Processing
#####################################################################
# Function to get gene expression data from TCGA via recount3
get_expression_data <- function(cancer_type, gene_name) {
  cache_file <- file.path("cache", paste0("expression_data_", cancer_type, "_", gene_name, ".rds"))
  
  # Check cache first
  if (file.exists(cache_file)) {
    message("Loading cached expression data...")
    return(readRDS(cache_file))
  }
  
  message("Getting expression data from recount3...")
  projects <- available_projects()
  
  # Find specific project in recount3
  project_info <- subset(projects, 
                        project == toupper(cancer_type) &
                        file_source == "tcga" &
                        project_type == "data_sources")
  
  if (nrow(project_info) != 1) {
    stop(sprintf("Could not find unique project for %s", cancer_type))
  }
  
  message("Creating RSE object...")
  rse_gene <- create_rse(
    project_info[1, ],
    type = "gene",
    annotation = "gencode_v26"
  )
  
  # Extract expression for specific gene
  gene_idx <- which(rowData(rse_gene)$gene_name == gene_name)[1]
  if (is.na(gene_idx)) {
    stop(sprintf("Gene %s not found in dataset", gene_name))
  }
  
  # Create data frame with expression values
  expression_data <- data.frame(
    case_id = substr(colData(rse_gene)$tcga.tcga_barcode, 1, 12),
    expression = assays(rse_gene)$counts[gene_idx, ]
  )
  
  # Handle duplicate samples by averaging
  if (any(duplicated(expression_data$case_id))) {
    expression_data <- expression_data %>%
      group_by(case_id) %>%
      summarize(expression = mean(expression, na.rm = TRUE)) %>%
      ungroup()
  }
  
  saveRDS(expression_data, cache_file)
  return(expression_data)
}

In [25]:
#####################################################################
# PSI (Percent Spliced In) Data Retrieval and Processing
#####################################################################
# Function to calculate PSI values for specific exons
get_psi_data <- function(cancer_type, gene_name = "SRRM3") {
  cache_file <- file.path("cache", paste0("psi_data_", cancer_type, "_", gene_name, ".rds"))
  
  if (file.exists(cache_file)) {
    message("Loading cached PSI data...")
    return(readRDS(cache_file))
  }
  
  # Define genomic coordinates for SRRM3 exon 15
  SRRM3_INFO <- list(
    gene = list(
      name = "SRRM3",
      chr = "chr7",
      start = 76201896,
      end = 76287287
    ),
    exon15 = list(
      start = 76283524,
      end = 76283602,
      length = 79
    )
  )
  
  message("Getting junction data from recount3...")
  projects <- available_projects()
  
  # Find specific project
  project_info <- subset(projects, 
                        project == toupper(cancer_type) &
                        file_source == "tcga" &
                        project_type == "data_sources")
  
  if (nrow(project_info) != 1) {
    message("\nAvailable TCGA projects:")
    print(subset(projects, file_source == "tcga" & project_type == "data_sources"))
    stop("Could not find unique project for ", cancer_type)
  }
  
  # Get junction data
  message("Creating RSE object...")
  rse_jxn <- create_rse(
    project_info[1, ],
    type = "jxn",
    jxn_format = "UNIQUE",
    verbose = TRUE
  )
  
  junction_counts <- assay(rse_jxn)
  jxn_coords <- rowRanges(rse_jxn)
  
  # Find junctions in SRRM3 region
  message("Finding relevant junctions...")
  srrm3_region <- which(
    start(jxn_coords) >= (SRRM3_INFO$exon15$start - 10000) &
    end(jxn_coords) <= (SRRM3_INFO$exon15$end + 10000) &
    seqnames(jxn_coords) == "chr7"
  )
  
  if (length(srrm3_region) == 0) {
    stop("No junctions found in SRRM3 region")
  }
  
  message(sprintf("Found %d junctions in SRRM3 region", length(srrm3_region)))
  
  # Extract relevant junctions
  jxn_coords <- jxn_coords[srrm3_region]
  junction_counts <- junction_counts[srrm3_region, ]
  
  # Identify inclusion and exclusion junctions
  inclusion_jxns <- which(
    (abs(end(jxn_coords) - SRRM3_INFO$exon15$start) <= 5) |
    (abs(start(jxn_coords) - SRRM3_INFO$exon15$end) <= 5)
  )
  
  exclusion_jxns <- which(
    start(jxn_coords) < (SRRM3_INFO$exon15$start - 5) &
    end(jxn_coords) > (SRRM3_INFO$exon15$end + 5)
  )
  
  message(sprintf("Found %d inclusion and %d exclusion junctions", 
                 length(inclusion_jxns), length(exclusion_jxns)))
  
  if (length(inclusion_jxns) == 0 || length(exclusion_jxns) == 0) {
    stop("Could not find both inclusion and exclusion junctions")
  }
  
  # Calculate PSI values for each sample
  psi_data <- data.frame(
    case_id = colData(rse_jxn)$tcga.tcga_barcode,
    psi = sapply(seq_len(ncol(junction_counts)), function(i) {
      inclusion_reads <- sum(junction_counts[inclusion_jxns, i])
      exclusion_reads <- sum(junction_counts[exclusion_jxns, i])
      total_reads <- inclusion_reads + exclusion_reads
      
      if(total_reads >= 10) {  # Minimum read coverage threshold
        return((inclusion_reads / total_reads) * 100)
      } else {
        return(NA)
      }
    })
  )
  
  # Clean and format data
  psi_data$case_id <- substr(psi_data$case_id, 1, 12)
  
  # Remove NA values and handle duplicates
  psi_data <- psi_data %>%
    filter(!is.na(psi)) %>%
    group_by(case_id) %>%
    summarize(psi = mean(psi, na.rm = TRUE)) %>%
    ungroup()
  
  message(sprintf("Found %d samples with valid PSI values", nrow(psi_data)))
  
  saveRDS(psi_data, cache_file)
  return(psi_data)
}

In [26]:
#####################################################################
# Gene Information Retrieval
#####################################################################
# Function to get gene coordinates and information from Ensembl
get_gene_info <- function(gene_name) {
  # Connect to Ensembl
  ensembl <- useMart("ensembl", dataset = "hsapiens_gene_ensembl")
  
  # Get gene coordinates and information
  gene_info <- getBM(
    attributes = c(
      "hgnc_symbol",
      "chromosome_name",
      "start_position",
      "end_position",
      "strand"
    ),
    filters = "hgnc_symbol",
    values = gene_name,
    mart = ensembl
  )
  
  if (nrow(gene_info) == 0) {
    stop(paste("Could not find gene info for", gene_name))
  }
  
  # Handle multiple entries
  if (nrow(gene_info) > 1) {
    message("Multiple entries found for ", gene_name, ", using first entry")
    gene_info <- gene_info[1, ]
  }
  
  # Standardize chromosome format
  if (!grepl("^chr", gene_info$chromosome_name)) {
    gene_info$chromosome_name <- paste0("chr", gene_info$chromosome_name)
  }
  
  return(gene_info)
}

# Analysis

In [32]:
# Set working directory
setwd("/beegfs/scratch/ric.broccoli/kubacki.michal/SRF_SRRM3/TCGA")


In [29]:
CANCER_TYPES <- c("ACC", "UVM", "SKCM", "LLG", "GBM")

In [30]:
# Set the cancer type you want to analyze
CANCER_TYPE <- "GBM"  # Change this to analyze different cancer types

# Analysis parameters
ANALYSIS_PARAMS <- list(
  gene = "SRRM3",
  data_type = "PSI",  # or "expression"
  grouping_method = "quartile"
)

# Print configuration
cat("Analysis Configuration:\n")
cat("Cancer Type:", CANCER_TYPE, "\n")
cat("Gene:", ANALYSIS_PARAMS$gene, "\n")
cat("Data Type:", ANALYSIS_PARAMS$data_type, "\n")
cat("Grouping Method:", ANALYSIS_PARAMS$grouping_method, "\n")

Analysis Configuration:
Cancer Type: GBM 
Gene: SRRM3 
Data Type: PSI 
Grouping Method: quartile 


In [36]:
# Source all functions from the original script
source("Multi_Cancer_Survival_Analysis.R")

# Reload the functions by detaching and re-sourcing
if ("package:Multi_Cancer_Survival_Analysis" %in% search()) {
  detach("package:Multi_Cancer_Survival_Analysis", unload = TRUE)
}
source("Multi_Cancer_Survival_Analysis.R")

# Get clinical data
clinical_data <- get_clinical_data(CANCER_TYPE)

# Display summary of clinical data
summary(clinical_data)

# Plot basic survival curve for all patients
fit_all <- survfit(Surv(overall_survival, deceased) ~ 1, data = clinical_data)
ggsurvplot(fit_all, 
           data = clinical_data,
           title = paste("Overall survival in", CANCER_TYPE),
           risk.table = TRUE)

Running in single-core mode

Running in single-core mode

Getting clinical data for GBM



ERROR: Error in value[[3L]](cond): Error retrieving clinical data for GBM: Supplied 617 items to be assigned to 1175 items of column 'submitter_id'. If you wish to 'recycle' the RHS please use rep() to make this intent clear to readers of your code.
